# Multi Query Retrieval

This method can automatically use a ChatModel to make slight variations of your initial query to help attempt overcome any issues with cosine similarity distances, this allows you to phrase things more like a general query question rather than a pure document similarity look-up.

In [6]:
from langchain_chroma import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [2]:
# Search Wikipedia and load relevant documents.. Each page is considered as a single document.
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

print(f"Length of all the documents - {len(documents)}")

/opt/homebrew/Caskroom/miniconda/base/envs/genai/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/homebrew/Caskroom/miniconda/base/envs/genai/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Length of all the documents - 24


In [3]:
# Split into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents=documents)

Created a chunk of size 525, which is longer than the specified 500
Created a chunk of size 588, which is longer than the specified 500


In [4]:
# OpenAI for Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [5]:
# Create the persistent ChromaDB
db = Chroma.from_documents(documents=docs,embedding=embeddings,persist_directory='some_data/mk_utra/')

In [7]:
# Use ChatModel for MultiQuery
question = "When was this declassified?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=llm)

In [8]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [11]:
unique_docs = retriever_from_llm.invoke(input=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of the declassification of this information?', '2. Can you provide the timeline for when this was declassified?', '3. At what point in time was this information made public?']


In [12]:
print(f"Length of the returned topics - {len(unique_docs)}")

Length of the Generated queries - 7


In [13]:
print(unique_docs[0].page_content)

== Background ==
In 1974, a New York Times article was published that accused the CIA of illegal operations committed against US citizens. Authored by Seymour M. Hersh, it documented an intelligence operation against the anti-war movement, as well as "break-ins, wiretapping and the surreptitious inspection of mail" conducted since the 1950s. According to former CIA Official Cord Meyer, these disclosures "Convinced large sections of the American public that the CIA had become a domestic Gestapo and stimulated an overwhelming demand for the wide-ranging congressional investigations that were to follow."
Hersh had been tipped off to the possibility of an "in house operation" by an unidentified member of the CIA in spring of 1974. He embarked on an investigation, speaking to sources that included CIA Chief of Counterintelligence James Angleton. Although he was not aware of its existence, Hersh uncovered much information that had been documented in the "Family Jewels", a report ordered by D